# 02 - Portfolios

In this notebook you can find simple examples that showcase how to programatically create portfolios.

In [1]:
import sys
sys.path.insert(0, '../')
%reload_ext autoreload
%autoreload 2
%pip install seaborn

You should consider upgrading via the '/home/saheru/Documents/Programming/Finance/IBKR/ibeam/env/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


## Start server

The `Server` class is the interface to perform HTTPS requests to acquire data regarding instruments, like stocks.

In [2]:
import time
import logging

import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt

from modules.server import Server
from modules.instruments import Portfolio

In [3]:
srv = Server()
srv.start()

# the following silences the logging output
logging.getLogger('ibkr-algotrading').setLevel(logging.ERROR)

Once started, we can request information on any stock we want simply by accesing items in `srv`. These will be cached, so only the first access will have significant delay.

First, it is a good idea to know which is our balance,

In [4]:
srv.balance

(1000000.0, 'USD')

> NOTE: with a simulated IBKR Paper Account you start with 1'000'000 USD, which you can use to test out trading.

## Create Portfolio

we can now request some stocks and create a portfolio with them,

In [5]:
stocks = srv[['SPY', 'XOM', 'MSFT', 'AAPL', 'TSLA'], '1y', '1d']

portfolio = Portfolio(wealth=100, stocks=stocks)

The `wealth` argument is in the same currency as our account's balance.

We can now check how much fraction of stock is allocated for each of the stocks.

Of course, right now we haven't bought any stock so it's all zero,

In [6]:
portfolio.alloc

array([0., 0., 0., 0., 0.])

Similarly, our distribution of wealth accross the stocks looks like an all-zero array, and
this is the only time it will be like so before we perform our first order.

In any other case the "weights" will sum up to 1.

In [7]:
portfolio.weights

array([0., 0., 0., 0., 0.])

we finally add the portfolio to the server so it knows about its existance,

In [8]:
srv.add_portfolio(portfolio)

## Place an order on our portfolio

To prevent accidentally losing all of our money, we will trade under a simulated context in which orders are only previewed but we still get whole information on what would happen,

In [9]:
# distribute the portfolio equally accross stocks
weights = np.ones(5) / 5.0

with srv.simulated():
    # 1st order
    print('1st order')
    portfolio.order(weights)

    # the wealth of the portfolio decreases since the
    # transactions take a commission
    print('Portfolio wealth: ', portfolio.wealth)
    print('Wealth + commission: ', portfolio.wealth + portfolio.commission)

    # 2nd order
    # we will put all our money on SPY
    weights[0] = 1.0
    weights[1:] = 0.0
    print('2nd order')
    portfolio.order(weights)

    print('Portfolio wealth: ', portfolio.wealth)
    print('Wealth + commission: ', portfolio.wealth + portfolio.commission)
    
    # we can also list all orders that have happened on this portfolio.
    # the stored values are the weights, or distribution of wealth
    # accross the stocks. So, 0.2 on SPY means 20% of the portfolio's
    # wealth is in the form of SPY's shares at that given time.
    print('Portfolio orders:')
    print(portfolio.orders)

1st order
Portfolio wealth:  99.04165200000001
Wealth + commission:  100.0
2nd order
Portfolio wealth:  98.04990300000001
Wealth + commission:  100.0
Portfolio orders:
                            SPY  XOM  MSFT  AAPL  TSLA
2021-08-29 20:07:00.345476  0.2  0.2   0.2   0.2   0.2
2021-08-29 20:07:03.587626  1.0  0.0   0.0   0.0   0.0


Note that after we exit the simulated context, the whole state of the server will be restored, meaning all orders in the
portfolio will disappear and it will return to its previous state.

In [10]:
# after exiting the simulation, the portfolio does not have any orders
print('Portfolio orders:')
print(portfolio.orders)

Portfolio orders:
Empty DataFrame
Columns: []
Index: []
